# Functional Dependence
In this notebook we will check the functional dependence of the purity indicator and bond orders.

In [ ]:
from os.path import join
geom = "renumber"
functionals = ["PBE", "BLYP", "HF", "B3LYP", "PBE0", "M06L", "SCAN"]

markers = {}
msize = {}
markers["PBE"] = "o"
markers["BLYP"] = "*"
markers["HF"] = "d"
markers["B3LYP"] = "v"
markers["PBE0"] = "+"
markers["M06L"] = "x"
markers["SCAN"] = "."

msize["PBE"] = 12
msize["BLYP"] = 12
msize["HF"] = 12
msize["B3LYP"] = 12
msize["PBE0"] = 16
msize["M06L"] = 16
msize["SCAN"] = 8

bset = "pcseg-1"

Read in the geometry.

In [ ]:
from pyntchem.io import read_pdb
with open(join("input", geom + ".pdb")) as ifile:
    sys = read_pdb(ifile)

Basis set.

In [ ]:
from pyntchem.basis import BasisSet
basis = BasisSet(bset, sys.get_symlookup())

Guess matrix comes from the calculations of basis dependence.

In [ ]:
alp = join("work-basis", basis.name, basis.name + ".DensAlp.mtx")

In [ ]:
from pyntchem.calculator import JobscriptCalculator
calc = JobscriptCalculator(computer="Spring", skip=True, verbose=True)
args = {"nodes": 1, "tasks_per_node": 4, "omp": 9,  "queue": "winter2"}

Iterate over functions.

In [ ]:
from pyntchem.preprocessing import put_guess_matrix
from contextlib import suppress
from pyntchem.inputfile import Inputfile
from time import sleep

# Note the different grids for metaggas
for fun in functionals:
    inp = Inputfile()
    if fun == "PBE":
        inp.set_basic_dft(xtype="pbe", ctype="pbe")
        inp.set_dft_prune_grid(50, 194)
    elif fun == "BLYP":
        inp.set_basic_dft(xtype="b88", ctype="lyp")
        inp.set_dft_prune_grid(50, 194)
    elif fun == "HF":
        inp.set_basic_rhf()
    elif fun == "B3LYP":
        inp.set_basic_dft(xctype="B3LYP")
        inp.set_dft_prune_grid(50, 194)
    elif fun == "PBE0":
        inp.set_custom_dft({"pbe": 0.75}, {"pbe": 1.0}, 0.25)
        inp.set_dft_prune_grid(50, 194)
    elif fun == "M06L":
        inp.set_basic_dft(xctype="M06L")
        inp.set_dft_prune_grid(75, 302)
    elif fun == "MN12L":
        inp.set_basic_dft(xctype="MN12L")
        inp.set_dft_prune_grid(75, 302)
    elif fun == "SCAN":
        inp.set_basic_dft(xctype="SCAN")
        inp.set_dft_prune_grid(75, 302)
    else:
        print(fun)
        break

    inp.set_scf_guess("readdens")
    inp["scf"].pulayperiod = 2
    inp["scf"].facdamp = 0.95
    inp["scf"].writeanal = True
    inp["scf"].vshift = 0

    inp["int2"].prelinkjthreshold = 1e-8
    inp["int2"].prelinkkthreshold = 1e-4
    
    with suppress(IOError):
        put_guess_matrix(join("work-func"), 
                         fun, dens_alp_file=alp)
    
    calc.run(sys, inp, basis, name=fun, 
             run_dir=join("work-func"),  **args)
    
while not calc.check_results(): sleep(10.0)
logfiles = {x: y.log for x, y in calc.calculations.items()}

Post-process to look at the purity values.

In [ ]:
from pyntchem.postprocessing import NTChemTool
tool = NTChemTool()

purities = {}
for f in functionals:
    log = logfiles[f]
    purities[f] = tool.run_compute_purity(sys, log)

Also bond order values.

In [ ]:
bond_orders = {}
for f in functionals:
    log = logfiles[f]
    bond_orders[f] = tool.fragment_bond_order(sys, list(sys), list(sys), log)

Read in BigDFT results.

In [ ]:
from pickle import load
with open("bigdft.cache", "rb") as ifile:
    purities["BigDFT-PBE"], bond_orders["BigDFT-PBE"] = load(ifile)

Plot purity values.

In [ ]:
order = sorted(sys, key=lambda x: int(x.split(":")[1]))

In [ ]:
from matplotlib import pyplot as plt
fig, axs = plt.subplots(1, 1, figsize=(5, 4))

for f in functionals:
    axs.plot([purities[f][k] for k in order], label=f,
             marker=markers[f], markersize=msize[f])
    
axs.plot([purities["BigDFT-PBE"][k] for k in order], 
         label="BigDFT-PBE", linestyle="--", color='k',
         linewidth=2)
    
axs.legend(loc="lower left", ncol=2)
axs.set_ylim(-0.06, 0)
axs.set_xticks(range(len(order)))
axs.set_xticklabels(order, rotation=90)
axs.tick_params(axis='both', which='major', labelsize=16)
axs.set_ylabel("Purity Indicator", fontsize=18)
fig.tight_layout()
fig.savefig("functional-pi.png", dpi=600)

Bond order.

In [ ]:
relevant = []
for k1, v1 in bond_orders["BigDFT-PBE"].items():
    for k2, v2 in v1.items():
        if k1 == k2:
            continue
        if (k2, k1) in relevant:
            continue
        if "MOL" in k1:
            if v2 > 1e-2:
                relevant.append((k1, k2))
        else:
            if v2 > 5e-3:
                relevant.append((k1, k2))
relevant = sorted(relevant, key=lambda x: int(x[0].split(":")[1]))

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 5))

for f in functionals:
    axs.plot([bond_orders[f][k[0]][k[1]] for k in relevant], label=f,
              marker=markers[f], markersize=msize[f])
    
axs.plot([bond_orders["BigDFT-PBE"][k[0]][k[1]] for k in relevant], 
         label="BigDFT-PBE", linestyle="--", color='k', linewidth=2)
    
axs.legend(loc="upper left", ncol=2)
axs.set_xticks(range(len(relevant)))
axs.set_xticklabels(["-".join(x) for x in relevant], rotation=90)
axs.set_ylim(5e-3, 5)
axs.set_yscale("log")

axs.tick_params(axis='both', which='major', labelsize=16)
axs.set_ylabel("Fragment Bond Order", fontsize=18)
fig.tight_layout()
fig.savefig("functional-fbo.png", dpi=600)